In [8]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
from scipy.stats import norm, skew
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

## scikit modeling libraries
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                             GradientBoostingClassifier, ExtraTreesClassifier,
                             VotingClassifier)

from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict,
                                     StratifiedKFold, learning_curve)

## Load metrics for predictive modeling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

# Preprocessing
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import LabelEncoder

## Warnings and other tools
import itertools
import warnings
warnings.filterwarnings("ignore")

# Funciones
from funciones import rmsle_cv

In [9]:
#Cargar el archivo csv
path = 'C:/THE BRIDGE_GIT/Machine_Learning/anime_score_detection_model_regresion/src/'
train = pd.read_csv(path + "data/processed/anime.csv")

In [10]:
train.rename(columns={'Unnamed: 0':'anime_id'}, inplace=True)

In [11]:
train.head(2)

,anime_id,MAL_ID,Genres,Type,Episodes,Studios,Rating,Start_year,Start_season,Score,range_episodes,Genres_Split,Studios_Split
0,0,1,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,Sunrise,R - 17+ (violence & profanity),1998,Spring,8.78,[25 - 37],"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",['Sunrise']
1,1,5,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,Bones,R - 17+ (violence & profanity),2001,Summer,8.39,[1 - 13],"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",['Bones']


In [12]:
#Eliminar columnas que no se usarán para el análisis
train.drop(['Genres_Split','Studios_Split','Start_year'] , axis = 1 , inplace = True)

### Genres

In [13]:
cleaning = train["Genres"].str.split(", ",12, expand=True)
cleaning.columns = ['Genre_0', 'Genre_1','Genre_2', 'Genre_3','Genre_4', 'Genre_5','Genre_6', 'Genre_7','Genre_8', 'Genre_9', 'Genre_10', 'Genre_11', 'Genre_12']
train_1 = pd.concat([train['anime_id'], cleaning], axis=1)

In [14]:
train_unpivoted = train_1.melt(id_vars=['anime_id'], var_name='Type_Genre', value_name='Genre')
train_unpivoted=train_unpivoted.assign(Value=1)
#Eliminar columnas que no se usarán para el análisis
train_unpivoted.drop(['Type_Genre'] , axis = 1 , inplace = True)
train_unpivoted.head(3)

,anime_id,Genre,Value
0,0,Action,1
1,1,Action,1
2,2,Action,1


In [15]:
train_pivoted = train_unpivoted.pivot_table(index=['anime_id'],columns=['Genre'],aggfunc='count',fill_value=0)
train_pivoted.columns = train_pivoted.columns.droplevel(0) #remove amount
train_pivoted = train_pivoted.reset_index().rename_axis(None, axis=1)
train_pivoted.head(3)

,anime_id,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [16]:
train_2 = pd.merge(train, train_pivoted, on='anime_id', how='outer')
#Eliminar columnas que no se usarán para el análisis
train_2.drop(['Genres'] , axis = 1 , inplace = True)
train_2.head(2)

,anime_id,MAL_ID,Type,Episodes,Studios,Rating,Start_season,Score,range_episodes,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0,1,TV,26,Sunrise,R - 17+ (violence & profanity),Spring,8.78,[25 - 37],1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1,5,Movie,1,Bones,R - 17+ (violence & profanity),Summer,8.39,[1 - 13],1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Studios

In [17]:
cleaning = train["Studios"].str.split(", ",6, expand=True)
cleaning.columns = ['Studio_0', 'Studio_1','Studio_2', 'Studio_3','Studio_4', 'Studio_5','Studio_6']
train_1 = pd.concat([train['anime_id'], cleaning], axis=1)

In [18]:
train_unpivoted = train_1.melt(id_vars=['anime_id'], var_name='Type_Studios', value_name='Studio')
train_unpivoted=train_unpivoted.assign(Value=1)
#Eliminar columnas que no se usarán para el análisis
train_unpivoted.drop(['Type_Studios'] , axis = 1 , inplace = True)
train_unpivoted.head(3)

,anime_id,Studio,Value
0,0,Sunrise,1
1,1,Bones,1
2,2,Madhouse,1


In [19]:
train_pivoted = train_unpivoted.pivot_table(index=['anime_id'],columns=['Studio'],aggfunc='count',fill_value=0)
train_pivoted.columns = train_pivoted.columns.droplevel(0) #remove amount
train_pivoted = train_pivoted.reset_index().rename_axis(None, axis=1)
train_pivoted.head(3)

In [ ]:
train_3 = pd.merge(train_2, train_pivoted, on='anime_id', how='outer')
#Eliminar columnas que no se usarán para el análisis
train_3.drop(['Studios'] , axis = 1 , inplace = True)
train_3.head(2)

,anime_id,Type,Episodes,Rating,Start_season,Score,Action,Adventure,Cars,Comedy,...,feel.,helo.inc,iDRAGONS Creative Studio,ixtl,l-a-unch・BOX,monofilmo,pH Studio,production doA,teamKG,ufotable
0,0,TV,26,R - 17+ (violence & profanity),Spring,8.78,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Movie,1,R - 17+ (violence & profanity),Summer,8.39,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Otras variables categóricas

In [ ]:
train_3.dtypes

anime_id           int64
Type              object
Episodes           int64
Rating            object
Start_season      object
                   ...  
monofilmo          int64
pH Studio          int64
production doA     int64
teamKG             int64
ufotable           int64
Length: 686, dtype: object

In [ ]:
# Para convertir las categóricas en binarias debemos pasarla primero a booleanas
# evaluando si son del tipo object o category
categorical_feature = (train_3.dtypes == "category") | (train_3.dtypes == object)
categorical_feature

anime_id          False
Type               True
Episodes          False
Rating             True
Start_season       True
                  ...  
monofilmo         False
pH Studio         False
production doA    False
teamKG            False
ufotable          False
Length: 686, dtype: bool

In [ ]:
categorical_cols = train_3.columns[categorical_feature].tolist()
categorical_cols

['Type', 'Rating', 'Start_season']

In [ ]:
for c in categorical_cols:
    lbl = LabelEncoder() 
    lbl.fit(list(train_4[c].values)) 
    train_4[c] = lbl.transform(list(train_4[c].values))


In [ ]:
# Ver un regisro
is_male = train_4.loc[:, 'anime_id'] == 1
df_male = train_4.loc[is_male]
df_male.head()

,anime_id,Type,Rating,Start_season,Score,Action,Adventure,Cars,Comedy,Dementia,...,helo.inc,iDRAGONS Creative Studio,ixtl,l-a-unch・BOX,monofilmo,pH Studio,production doA,teamKG,ufotable,range_episodes
1,1,0,3,2,8.39,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_4.dtypes

anime_id            int64
Type                int32
Rating              int32
Start_season        int32
Score             float64
                   ...   
pH Studio           int64
production doA      int64
teamKG              int64
ufotable            int64
range_episodes      int32
Length: 686, dtype: object

## Seleccionar Variables

In [ ]:
# X = train_4.drop(['Score','anime_id'], axis=1)
X = train_4.drop(['Score'], axis=1)
y = train_4['Score']

## Entrenar el modelo

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import lightgbm as lgb

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
## print informacion
print("X_train shape:", X_train.shape, "| X_test shape:", X_test.shape)
print("y_train mean:", round(np.mean(y_train),2), "| y_test mean:", round(np.mean(y_test),2))

X_train shape: (7266, 685) | X_test shape: (1817, 685)
y_train mean: 6.74 | y_test mean: 6.74


In [43]:
# n_folds = 5

# def rmsle_cv(model):
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
#     rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
#     return(rmse)

**Linear Reggression**

In [44]:
#Linear Regression
lm = LinearRegression()
lm.fit(X_train, y_train)
score = rmsle_cv(lm)
print("\nLinear score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Linear score: 2345597.0320 (3921564.2368)



**Lasso Regression**

In [45]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.6787 (0.0186)



**Elastic Net Regression**

In [46]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.6779 (0.0185)



**Kernel Ridge Regression**

In [47]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Kernel Ridge score: 0.7303 (0.0873)



**Gradient Boosting Regression**

In [48]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 0.6271 (0.0141)



**XGBoost Regression**

In [49]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1, verbosity=0)
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 0.6107 (0.0128)



**LGBM**

In [50]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11

# Fitting the best model and getting the predictions array

In [51]:
model_xgb.fit(X_train,y_train)
xgb_pred = model_xgb.predict(X_test)


In [56]:
result_xgb = pd.DataFrame({'Y_TEST':y_test.values,'Y_PRED' : pd.Series(xgb_pred).values})
result_xgb

In [58]:
GBoost.fit(X_train,y_train)
GBoost_pred = GBoost.predict(X_test)

In [62]:
lasso.fit(X_train,y_train)
lasso_pred = lasso.predict(X_test)

In [64]:
ENet.fit(X_train,y_train)
ENet_pred = ENet.predict(X_test)

In [67]:
KRR.fit(X_train,y_train)
KRR_pred = KRR.predict(X_test)

### Métricas del modelo

In [54]:
from sklearn.metrics import mean_absolute_error as mae

In [55]:
mae(y_test, xgb_pred)

0.4658208870990452

In [59]:
mae(y_test, GBoost_pred)

0.4778890770272293

In [63]:
mae(y_test, lasso_pred)

0.5241479906963852

In [65]:
mae(y_test, ENet_pred)

0.5235683285770798

In [68]:
mae(y_test, KRR_pred)

0.5171850364767213